In [ ]:
#Importing necessary libraries
import nltk
import pandas as pd
from textblob import Word
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split

In [ ]:
#Loading the dataset
data = pd.read_csv('data.csv')

In [ ]:
data.columns

Index(['Sentence', 'Sentiment'], dtype='object')

In [ ]:
#Pre-Processing the text
def cleaning(df, stop_words):
    #df['sentences'] = df['Sentence'].apply(lambda x: ' '.join(x.lower() for x in x.split()))
    df['Sentence'] = df['Sentence'].apply(lambda x: ' '.join([Word(word).lemmatize() for word in x.split()]))
    # Replacing the digits/numbers
    df['Sentence'] = df['Sentence'].str.replace('d', '')
    # Removing stop words
    df['Sentence'] = df['Sentence'].apply(lambda x: ' '.join(x for x in x.split() if x not in stop_words))
    # Lemmatization
    df['Sentence'] = df['Sentence'].apply(lambda x: ' '.join([Word(x).lemmatize() for x in x.split()]))
    return df

In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [ ]:
#stop_words = stopwords.words('english')
data_cleaned = cleaning(data, stop_words)

In [ ]:
data_cleaned.columns

Index(['Sentence', 'Sentiment'], dtype='object')

In [ ]:
#Generating Embeddings using tokenizer
tokenizer = Tokenizer(num_words=500, split=' ')
tokenizer.fit_on_texts(data_cleaned['Sentence'].values)
#X = tokenizer.texts_to_sequences(data_cleaned['sentences'].values)
#y = pad_sequences(X)
reviews = data['Sentence'].values
labels = data['Sentiment'].values
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

In [ ]:
#X
#reviews
encoded_labels


array([2, 0, 2, ..., 1, 1, 2])

In [ ]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews, encoded_labels, stratify = encoded_labels)


In [ ]:
# Hyperparameters of the model
vocab_size = 3000 # choose based on statistics
oov_tok = ''
embedding_dim = 100
max_length = 200 # choose based on statistics, for example 150 to 200
padding_type='post'
trunc_type='post'
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)
# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [ ]:
# model initialization
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 100)          300000    
                                                                 
 bidirectional (Bidirection  (None, 128)               84480     
 al)                                                             
                                                                 
 dense_2 (Dense)             (None, 24)                3096      
                                                                 
 dense_3 (Dense)             (None, 1)                 25        
                                                                 
Total params: 387601 (1.48 MB)
Trainable params: 387601 (1.48 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
num_epochs = 5
history = model.fit(train_padded, train_labels,
                    epochs=num_epochs, verbose=1,
                    validation_split=0.1)

Epoch 1/5
124/124 [==============================] - 37s 262ms/step - loss: -4.9103 - accuracy: 0.5370 - val_loss: -17.5953 - val_accuracy: 0.5103
Epoch 2/5
124/124 [==============================] - 29s 238ms/step - loss: -21.7248 - accuracy: 0.5386 - val_loss: -47.4406 - val_accuracy: 0.5103
Epoch 3/5
124/124 [==============================] - 30s 241ms/step - loss: -47.3406 - accuracy: 0.5386 - val_loss: -92.5125 - val_accuracy: 0.5103
Epoch 4/5
124/124 [==============================] - 31s 247ms/step - loss: -84.0136 - accuracy: 0.5386 - val_loss: -151.8882 - val_accuracy: 0.5103
Epoch 5/5
124/124 [==============================] - 29s 237ms/step - loss: -131.1712 - accuracy: 0.5386 - val_loss: -227.1140 - val_accuracy: 0.5103


In [ ]:
prediction = model.predict(test_padded)
# Get labels based on probability 1 if p>= 0.5 else 0
pred_labels = []
for i in prediction:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
print("Accuracy of prediction on test set : ", accuracy_score(test_labels,pred_labels))

46/46 [==============================] - 7s 115ms/step
Accuracy of prediction on test set :  0.5359342915811088


In [ ]:
#ds_split, info = tfds.load("penguins/processed", split=['train[:20%]', 'train[20%:]'], as_supervised=True, with_info=True)

#ds_test = ds_split[0]
#ds_train = ds_split[1]
#assert isinstance(ds_test, tf.data.Dataset)

#print(info.features)
#df_test = tfds.as_dataframe(ds_test.take(5), info)
#print("Test dataset sample: ")
#print(df_test)

#df_train = tfds.as_dataframe(ds_train.take(5), info)
#print("Train dataset sample: ")
#print(df_train)

#ds_train_batch = ds_train.batch(32)